In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import pandas as pd
import numpy as np


from datetime import datetime as dt
from tqdm import tqdm
tqdm.pandas()

In [8]:
venues = pd.read_csv('data/venues.csv')
listings = pd.read_csv('data/listings.csv')
calendar = pd.read_csv('data/calendar.csv')

In [ ]:
def top_business(row, clusters):
    dist = clusters.apply(lambda x: geodesic((x['lat'],x['lon']), (row['latitude'], row['longitude'])).km,axis=1).sort_values(ascending=True).head(1)
    return str(dist[dist.index[0]])

def mega_dist(row, clusters):
    mega = cluster_df.iloc[cluster_df['num'].astype('int').argmax()]
    dist = geodesic((mega['lat'],mega['lon']), (row['latitude'], row['longitude'])).km
    return str(dist)

In [ ]:
cities = ['asheville', 'nashville', 'austin', 'new orleans', 'los angeles']
types = ['public_transport', 'night_life', 'minor_dest', 'major_dest', 'food']

In [ ]:
len(np.load('clusters/'+'public_transport'+'_'+'los angeles'+'_cluster_10.npy'))

In [ ]:
cluster_df.iloc[cluster_df['num'].astype('int').argmax()]

In [ ]:
cluster_df

In [ ]:
multi_owner = listings.groupby('host_id').filter(lambda x:len(x) > 1)
multi_owner['metropolitan'] = multi_owner['metropolitan'].replace('LA', 'los angeles')
multi_owner['metropolitan'] = multi_owner['metropolitan'].replace('new_orleans', 'new orleans')
distance_data = []
for city in cities:
    multi_owner_city = multi_owner[multi_owner['metropolitan'] == city]
    print(city)
    for type_dest in types:
        print(type_dest)
        cluster_np = np.load('clusters/'+type_dest+'_'+city+'_cluster_10.npy')
        if len(cluster_np) == 0:
            cluster_np = np.load('clusters/'+type_dest+'_'+city+'_cluster.npy')
        print('clusters/'+type_dest+'_'+city+'_cluster.npy')
        print(cluster_np)
#         if len(cluster_np) > 0:
#             cluster_df = pd.DataFrame(cluster_np, columns=['num','lat','lon','zipcode'])
#             multi_owner_city[type_dest] = multi_owner_city.apply(top_business,axis = 1, clusters=cluster_df)
#             multi_owner_city['mega_'+type_dest] = multi_owner_city.apply(mega_dist,axis = 1, clusters=cluster_df)
#         else:
#             multi_owner_city[type_dest] = np.nan
#             multi_owner_city['mega_'+type_dest] = np.nan
#     distance_data.append(multi_owner_city)



In [ ]:
new_dist = pd.concat(distance_data)

In [ ]:
new_dist.columns

In [ ]:
all_dist[['major_dest', 'mega_major_dest']] = new_dist[['major_dest', 'mega_major_dest']]

In [ ]:
import xgboost

In [ ]:
all_dist = pd.concat(distance_data)

In [ ]:
all_dist = pd.read_csv('all_dist.csv')

In [ ]:
all_dist

In [ ]:
all_dist.columns

In [ ]:
tree_data = all_dist_ratio[['accommodates', 
                     'bathrooms',
                     'bed_type', 
                     'bedrooms', 
                     'beds',
                     'cancellation_policy',
                     'metropolitan',
                     'property_type',
                     'review_scores_checkin',
                     'review_scores_cleanliness',
                     'review_scores_communication', 
                     'review_scores_location',
                     'review_scores_rating',
                     'review_scores_value', 
                     'room_type',
                     'public_transport',
                     'mega_public_transport',
                     'night_life', 'mega_night_life', 'minor_dest', 'mega_minor_dest',
                     'major_dest', 'mega_major_dest', 'food', 'mega_food',
                     'available',
                    'price']]

In [ ]:
tree_data['bed_type'] = tree_data['bed_type'].astype('category').cat.rename_categories([1, 2, 3, 4, 5])

In [ ]:
tree_data['metropolitan'] = tree_data['metropolitan'].astype('category').cat.rename_categories([1, 2, 3, 4, 5])
tree_data['property_type'] = tree_data['property_type'].astype('category').cat.rename_categories(list(range(35)))
tree_data['room_type'] = tree_data['room_type'].astype('category').cat.rename_categories(list(range(3)))
tree_data['cancellation_policy'] = tree_data['cancellation_policy'].astype('category').cat.rename_categories([1, 2, 3, 4, 5, 6])

In [ ]:
len(tree_data['room_type'].unique()) #= tree_data['bed_type'].astype('category').cat.rename_categories([1, 2, 3, 4, 5])

In [ ]:
tree_data

In [ ]:
tree_data[['public_transport',
                     'mega_public_transport',
                     'night_life', 'mega_night_life', 'minor_dest', 'mega_minor_dest',
                     'major_dest', 'mega_major_dest', 'food', 'mega_food']] = tree_data[['public_transport',
                     'mega_public_transport',
                     'night_life', 'mega_night_life', 'minor_dest', 'mega_minor_dest',
                     'major_dest', 'mega_major_dest', 'food', 'mega_food']].astype('float')

In [ ]:
old_data = tree_data[['major_dest', 'mega_major_dest']]

In [ ]:
tree_data.dtypes

In [ ]:
from xgboost import XGBRegressor
X = tree_data.drop('available', axis=1).to_numpy()
y = tree_data['available'].to_numpy()
model = XGBRegressor()
model.fit(X, y)
# feature importance


In [ ]:
feature_imp = np.array(model.feature_importances_)

In [ ]:
feature_imp[feature_imp.argsort()[-10:][::-1]]

In [ ]:
tree_data.drop('available', axis=1).columns[feature_imp.argsort()[-10:][::-1]]

In [ ]:
from matplotlib import pyplot
pyplot.bar(range(len(model.feature_importances_)), model.feature_importances_)
pyplot.show()

In [ ]:
list(tree_data.drop('available', axis=1).columns[feature_imp.argsort()[-10:][::-1]])

In [ ]:
import seaborn as sns

ax = sns.barplot(x=list(tree_data.drop('available', axis=1).columns[feature_imp.argsort()[-10:][::-1]]), y=feature_imp[feature_imp.argsort()[-10:][::-1]])
plt.xticks(rotation = 90)
plt.figure(figsize = (20, 10))

In [ ]:
arr.argsort()[-3:][::-1]

In [ ]:
all_dist.to_csv('all_dist.csv')

In [15]:
calendar['available'] = calendar['available'].replace('f', 0)
calendar['available'] = calendar['available'].replace('t', 1)

In [ ]:
len(calendar)

In [ ]:
calendar['available']

In [16]:
ratios = calendar.groupby(['listing_id'])['available'].sum()/calendar.groupby('listing_id')['available'].count()

In [18]:
ratios.to_csv('ratios.csv')

ModuleNotFoundError: No module named 'pandas.io.formats.csvs'

In [24]:
np.save('ratios.npy',[np.array(ratios.index),ratios.to_numpy()])

In [23]:
[ratios.index,ratios.to_numpy()]

[Int64Index([     109,      344,      941,     1078,     2265,     2404,
                 2732,     2864,     3021,     3152,
             ...
             19013292, 19013896, 19014529, 19016073, 19032418, 19034190,
             19035492, 19035679, 19037230, 19038571],
            dtype='int64', name='listing_id', length=50364),
 array([0.        , 0.        , 0.43561644, ..., 0.98082192, 0.27945205,
        0.38356164])]

In [ ]:
ratios.reset_index()

In [ ]:
all_dist_ratio = all_dist.merge(ratios, how='inner', left_on='id', right_on='listing_id')

In [ ]:
all_dist_ratio.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
all_dist_ratio.to_csv('all_dist_availablity.csv')

In [ ]:
all_dist.columns

In [11]:
all_dist_ratio['price'] = all_dist_ratio['price'].str.replace('$','').str.replace(',','').astype('float') 
all_dist_ratio['weekly_price'] = all_dist_ratio['weekly_price'].str.replace('$','').str.replace(',','').astype('float')

<ipython-input-11-15a5477df7bd>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  all_dist_ratio['price'] = all_dist_ratio['price'].str.replace('$','').str.replace(',','').astype('float')
<ipython-input-11-15a5477df7bd>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  all_dist_ratio['weekly_price'] = all_dist_ratio['weekly_price'].str.replace('$','').str.replace(',','').astype('float')


In [ ]:
all_dist_ratio.dtypes

In [ ]:
all_dist_ratio[['price', 'public_transport',
                     'mega_public_transport',
                     'night_life', 'mega_night_life', 'minor_dest', 'mega_minor_dest',
                     'major_dest', 'mega_major_dest', 'food', 'mega_food','available']].astype('float').corr()

In [ ]:
all_dist_ratio[['price', 'public_transport',
                     'mega_public_transport',
                     'night_life', 'mega_night_life', 'minor_dest', 'mega_minor_dest',
                     'major_dest', 'mega_major_dest', 'food', 'mega_food',
       'weekly_price', 'review_scores_value', 'available']].astype('float').corr(method='kendall')

In [12]:
correl = all_dist_ratio[['price','available','public_transport',
                     'mega_public_transport',
                     'night_life', 'mega_night_life', 'minor_dest', 'mega_minor_dest',
                     'major_dest', 'mega_major_dest', 'food', 'mega_food']].astype('float').corr(method='spearman')

In [9]:
all_dist_ratio = pd.read_csv('data/all_dist_availablity2.csv')

In [ ]:
all_dist_ratio

In [ ]:
correl

In [14]:
import seaborn as sns
sns.heatmap(correl,annot=True, fmt = ".2f", cmap = "coolwarm")
plt.xticks(rotation = 90)

plt.figure(figsize = (160, 80))
plt.savefig('cluster_dist.pdf')

AttributeError: module 'seaborn' has no attribute 'heatmap'

In [ ]:
import geopandas

In [ ]:
from scipy.stats import pearsonr
import pandas as pd

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 10)
    return pvalues



In [ ]:
vars_df = all_dist_ratio[all_dist_ratio['metropolitan'] == 'los angeles'][['price', 'public_transport',
                     'mega_public_transport',
                     'night_life', 'mega_night_life', 'minor_dest', 'mega_minor_dest',
                     'major_dest', 'mega_major_dest', 'food', 'mega_food',
       'weekly_price', 'review_scores_value', 'available']].astype('float')
calculate_pvalues(vars_df)

In [ ]:
venues['city'].unique()

In [ ]:
newo_food

In [ ]:
austin_major_dest

In [ ]:
ashe_major_dest


In [ ]:
night_life_cluster

In [ ]:
multi_owner_ashe.apply(top_business,axis = 1, clusters=minor_dest_cluster)

In [ ]:
major_dest_cluster


In [ ]:
multi_owner_ashe

In [ ]:
multi_owner.merge(calendar, how='inner', left_on='id', right_on='listing_id')